# Asset Allocation with Pytorch

 Goal of this notebook is to create a model to combine the signals from the various strategies to create an asset allocation neural network to outperform equally weighted, as Markowitz may not be appropriate as the signals may be to buy or sell the same asset therefore yielding a perfect correlation when active.

In [2]:
from loguru import logger
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import shutil
import time
from tqdm import tqdm
import typing as t

import viz_neural_network as viz_nn

In [3]:
from quantified_strategies import loss, ml_utils, plot_utils, strategy_utils, utils

2024-02-26 21:29:00.303 | INFO     | quantified_strategies.ml_utils:get_device:26 - Running on the CPU


In [4]:
CASH = "CASH"

In [5]:
ASSETS = ["SPY", "QQQ", CASH]
# ASSETS = ["SPY", "QQQ", CASH]
ASSETS

['SPY', 'QQQ', 'CASH']

In [6]:
LONG_OVERNIGHT_COSTS_DICT = {
    "SPY": 0.507559 / 508.3,
    "QQQ": 0.113025 / 437,
    "XLC": 0.020313 / 78.53,
    "XLE": 0.086145 / 86.25,
    "XLF": 0.040039 / 40.08,
    "XLI": 0.031045 / 120.00,
    "XLK": 0.053141 / 205.50,
    "XLP": 0.019255 / 74.47,
    "XLRE": 0.038212 / 38.28,
    "XLU": 0.015704 / 60.74,
    "XLY": 0.04701 / 181.00,
    "XLV": 0.038038 / 147.10,
}
SHORT_OVERNIGHT_COSTS_DICT = {
    "SPY": 0.127772 / 508.3,
    "QQQ": 0.089958 / 437,
    "XLC": 0.016167 / 78.53,
    "XLE": 0.021686 / 86.25,
    "XLF": 0.010079 / 40.08,
    "XLI": 0.024709 / 120.00,
    "XLK": 0.042296 / 205.50,
    "XLP": 0.015325 / 74.47,
    "XLRE": 0.009619 / 38.28,
    "XLU": 0.012499 / 60.74,
    "XLY": 0.037416 / 181.00,
    "XLV": 0.030275 / 147.10,
}

In [7]:
# Temporary Values
BORROWING_COSTS = [0 / 10_000 for _ in ASSETS]

## Dataset Collection

In [8]:
def get_data(assets: str | t.List[str], is_classification: bool = True) -> t.Tuple[pd.DataFrame, pd.DataFrame]:

    def get_y() -> pd.DataFrame:
        price_data = [strategy_utils.get_data(ticker=ticker, columns="Adj Close").to_frame(name=ticker) for ticker in assets if ticker != "CASH"]
        price_data = pd.concat(price_data, axis=1)
        return_data = price_data.pct_change()
        if "CASH" in assets:
            return_data["CASH"] = 0.0
        return_data = return_data.shift(-1)
        return_data = return_data.dropna()

        if is_classification:
            return (return_data > 0).astype(int)
        
        return return_data

    def get_X() -> pd.DataFrame:
        strategy_returns = pd.read_csv(f"outputs/strategy_returns.csv", index_col=0, header=[0, 1, 2])
        strategy_returns = strategy_returns.loc[:, strategy_returns.columns.get_level_values(2).isin(assets)]
        strategy_returns.index = pd.DatetimeIndex(strategy_returns.index)
        is_active = ~(strategy_returns.isna())
        is_active = is_active.astype(int)
        return is_active

    assets = assets if isinstance(assets, list) else [assets]

    # Get target variables: these are the returns from entering a position from close to close t+1
    y = get_y()
    
    # Get explanatory variables: these are the signals from the strategies indicating whether to buy or not
    X = get_X()

    X = X.loc[X.index.isin(X.index.intersection(y.index))]
    y = y.loc[y.index.isin(y.index.intersection(X.index))]

    X = X.sort_index()
    y = y.sort_index()

    return X, y

orig_X_total, orig_y_total = get_data(assets=ASSETS, is_classification=False)

In [9]:
print(f"{orig_X_total.shape = }")
print(f"{orig_y_total.shape = }")

orig_X_total.shape = (6073, 16)
orig_y_total.shape = (6073, 3)


In [10]:
TRADE_ID = "trade_id"
STRATEGY_ID = "strategy_id"


def group_trades(X: pd.DataFrame, y: pd.DataFrame) -> t.Tuple[pd.DataFrame, pd.DataFrame]:
    
    new_X = X.copy()
    new_y = y.copy()
    
    # Fetch trade id i.e. change in strategy activation
    new_X[STRATEGY_ID] = new_X.apply(lambda x: sum([v * 10 ** i for i, v in enumerate(x.values)]), axis=1)
    new_X[TRADE_ID] = (new_X[STRATEGY_ID].diff().abs().fillna(1.0) > 0).cumsum()
    
    # Copy trade id to target/asset return dataframe
    new_y[TRADE_ID] = new_X[TRADE_ID]
    
    # Find trade id to first trade date
    trades = new_y.reset_index().set_index(TRADE_ID)["Date"]
    trades = trades[~trades.index.duplicated()]
    trades_to_date_dict = trades.to_dict()
    
    # Find return for each trade id
    new_y = new_y.groupby(by=TRADE_ID)[ASSETS].apply(lambda ret: strategy_utils.get_cumulative_return(returns=ret, total=True))
    new_y.index = new_y.index.map(trades_to_date_dict)
    new_y.index.name = "Date"

    X = X.loc[X.index.isin(new_y.index)].copy()

    return X, new_y

orig_X_total, orig_y_total = group_trades(X=orig_X_total, y=orig_y_total)

In [11]:
orig_X_total.head(1)

event_trading       overnight_trading            seasonal_trading  \
              super_bowl     short_term_reversal     buy_when_yields_are_low   
                     SPY QQQ                 SPY QQQ                     SPY   
Date                                                                           
2000-01-03             0   0                   0   0                       0   

                                                                             \
               pay_day_strategy     santa_claus_strategy     september_bear   
           QQQ              SPY QQQ                  SPY QQQ            SPY   
Date                                                                          
2000-01-03   0                1   1                    1   1              0   

                                                                      
               tax_day_strategy     turn_around_tuesday_strategy      
           QQQ              SPY QQQ                          SPY QQQ  
Date                                                                  
2000-01-03   0                0   0                            0   0

In [12]:
orig_y_total.head(5)

,SPY,QQQ,CASH
Date,,,
2000-01-03,-0.039107,-0.068602,0.0
2000-01-04,-0.014310,-0.092422,0.0
2000-01-06,0.058075,0.123683,0.0
2000-01-07,-0.037092,0.018055,0.0
2000-01-24,0.011356,0.015007,0.0


In [13]:
assert orig_X_total.dropna().shape[0] == orig_X_total.shape[0]
assert orig_y_total.dropna().shape[0] == orig_y_total.shape[0]

## Create Pytorch Model

In [14]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim

### Define `device` to use when training model

In [15]:
DEVICE = ml_utils.get_device()
DEVICE

2024-02-26 21:29:04.121 | INFO     | quantified_strategies.ml_utils:get_device:26 - Running on the CPU


device(type='cpu')

In [16]:

class Net(nn.Module):

    DEFAULT_LAYER_SIZES: t.List[int] = [8, 16, 8]
    DEFAULT_ALLOW_NEGATIVE_WEIGHTS: bool = False

    # Used when more than one asset is being traded, enables leverage.
    DEFAULT_MAX_WEIGHT: float = 1.0
    DEFAULT_MIN_WEIGHT: float = 0.0
    
    # Model Type: used to save model
    MODEL_TYPE: str = "nn"
    
    def __init__(self, input_shape: int, output_shape: int, layer_sizes: t.List[int] = DEFAULT_LAYER_SIZES, 
                 allow_negative_weights: bool = DEFAULT_ALLOW_NEGATIVE_WEIGHTS, 
                 max_weight: float = None, min_weight: float = None):
        super().__init__()
        
        self.input_shape: int = input_shape
        self.output_shape: int = output_shape
        self.layer_sizes: t.List[int] = layer_sizes
        
        self.allow_negative_weights: bool = allow_negative_weights

        if (max_weight is None and min_weight is None) or self.output_shape == 1:
            max_weight = Net.DEFAULT_MAX_WEIGHT
            min_weight = Net.DEFAULT_MIN_WEIGHT
        elif max_weight is None and min_weight is not None:
            max_weight = -min_weight * (self.output_shape - 1) + 1
        elif max_weight is not None and min_weight is None:
            min_weight = -(max_weight - 1) / (self.output_shape - 1)
        else:
            pass

        # Assert MAX > MIN
        assert max_weight > min_weight, f"'max_weight' must be larger than 'min_weight': provided {max_weight = } and {min_weight = }"
        # Assert MAX + (n - 1) * MIN
        assert max_weight + (self.output_shape - 1) * min_weight == 1, f"'max_weight' plus (n - 1) * 'min_weight' should be equal to 1: " +\
            f"{max_weight} + {self.output_shape - 1} * {min_weight} = {max_weight + (self.output_shape - 1) * min_weight}"
        
        self.allow_negative_weights: bool = min_weight < 0
        self.max_weight: float = max_weight
        self.min_weight: float = min_weight
        self._max_weight: float = max_weight - 1 / self.output_shape
        self._min_weight: float = min_weight - 1 / self.output_shape
        
        last_shape = self.input_shape
        for i, layer_size in enumerate(self.layer_sizes):
            setattr(self, f"fc{i}", nn.Linear(last_shape, layer_size))
            # setattr(self, f"dropout{i}", nn.Dropout(p=0.2))
            last_shape = layer_size
        self.fc_output = nn.Linear(last_shape, self.output_shape)

    def forward(self, x):

        for i, _ in enumerate(self.layer_sizes):
            x = getattr(self, f"fc{i}")(x)
            # x = F.relu(x)
            # x = F.leaky_relu(x)
            x = F.elu(x)
            # x = getattr(self, f"dropout{i}")(x)

        x = self.fc_output(x)

        if self.output_shape == 1:
            if self.allow_negative_weights:
                # Boundaries: (-1, +1)
                output = F.tanh(x)
            else:
                # Boundaries: (0, +1)
                output = F.sigmoid(x)
        else:
            # Boundaries: (min_weight, max_weight), Sum: 1.0
            output = F.softmax(x, dim=1)
            output = (self._max_weight - self._min_weight) * output + self._min_weight + 1 / self.output_shape

        return output

    @staticmethod
    def translate(X: torch.Tensor, y: torch.Tensor, **kwargs) -> t.Tuple[torch.Tensor, torch.Tensor]:
        return X, y

    @staticmethod
    def load(input_shape: int, output_shape: int, name: str = "latest"):
        
        PATH = Path(os.getcwd())
        model_dict = torch.load(PATH / f"outputs/models/{Net.MODEL_TYPE}-model-{name}-state.dict")
        
        net = Net(input_shape=input_shape, output_shape=output_shape)
        net.load_state_dict(model_dict)
        net.eval()
        
        return net

    def save(self, name: str) -> None:
        PATH = Path(os.getcwd())
        torch.save(self.state_dict(), PATH / f"outputs/models/{self.MODEL_TYPE}-model-{name}-state.dict")
        shutil.copy(PATH / f"outputs/models/{self.MODEL_TYPE}-model-{name}-state.dict", 
                    PATH / f"outputs/models/{self.MODEL_TYPE}-model-latest-state.dict")
        return


def example():

    # Define input and output sizes for neural network
    INPUT_SHAPE = 10
    OUTPUT_SHAPE = 2
    print(f"Input Shape = {INPUT_SHAPE}, Output Shape = {OUTPUT_SHAPE}")

    # Generate example data
    N_SAMPLES = 10
    X_sample = torch.randn(N_SAMPLES, INPUT_SHAPE)
    y_sample = torch.randn(N_SAMPLES, 1)
    print(f"{X_sample.shape = }, {y_sample.shape = }")
    
    X_sample_translated, y_sample_translated = Net.translate(X=X_sample, y=y_sample)
    print(f"{X_sample_translated.shape = }, {y_sample_translated.shape = }")

    X_sample_translated = X_sample_translated.to(device=DEVICE)
    y_sample_translated = y_sample_translated.to(device=DEVICE)

    # Initiate Network
    my_net = Net(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, layer_sizes=[5, 10, 5], allow_negative_weights=False).to(device=DEVICE)
    output = my_net.forward(x=X_sample_translated)
    print(f"{output.shape = }")
    print(f"{output = }")

    my_net = Net(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, layer_sizes=[8, 16, 8], allow_negative_weights=True).to(device=DEVICE)
    output = my_net.forward(x=X_sample_translated)
    print(f"{output.shape = }")
    print(f"{output = }")

    my_net = Net(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, layer_sizes=[8, 16, 8], 
                 allow_negative_weights=True, max_weight=2.0, min_weight=-1.0).to(device=DEVICE)
    output = my_net.forward(x=X_sample_translated)
    print(f"{output.shape = }")
    print(f"{output = }")
    
    return

example()

Input Shape = 10, Output Shape = 2
X_sample.shape = torch.Size([10, 10]), y_sample.shape = torch.Size([10, 1])
X_sample_translated.shape = torch.Size([10, 10]), y_sample_translated.shape = torch.Size([10, 1])
output.shape = torch.Size([10, 2])
output = tensor([[0.5602, 0.4398],
        [0.5438, 0.4562],
        [0.5552, 0.4448],
        [0.5459, 0.4541],
        [0.5879, 0.4121],
        [0.5600, 0.4400],
        [0.5489, 0.4511],
        [0.5646, 0.4354],
        [0.5583, 0.4417],
        [0.5514, 0.4486]], grad_fn=<AddBackward0>)
output.shape = torch.Size([10, 2])
output = tensor([[0.6223, 0.3777],
        [0.5901, 0.4099],
        [0.5866, 0.4134],
        [0.6078, 0.3922],
        [0.6115, 0.3885],
        [0.6078, 0.3922],
        [0.5984, 0.4016],
        [0.5965, 0.4035],
        [0.5983, 0.4017],
        [0.6439, 0.3561]], grad_fn=<AddBackward0>)
output.shape = torch.Size([10, 2])
output = tensor([[0.5224, 0.4776],
        [0.4812, 0.5188],
        [0.5491, 0.4509],
        [0.

In [17]:
INPUT_SHAPE = 10
OUTPUT_SHAPE = 2
my_net = Net(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, layer_sizes=[5, 10, 5, 4], allow_negative_weights=False).to(device=DEVICE)
my_net

Net(
  (fc0): Linear(in_features=10, out_features=5, bias=True)
  (fc1): Linear(in_features=5, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=5, bias=True)
  (fc3): Linear(in_features=5, out_features=4, bias=True)
  (fc_output): Linear(in_features=4, out_features=2, bias=True)
)

### Translate Data to Correct Format

In [18]:
X_total, y_total = ml_utils.convert_data_to_tensors(X=orig_X_total, y=orig_y_total)
X_total, y_total = Net.translate(X=X_total, y=y_total)
X_train, X_test, y_train, y_test = ml_utils.split_data(X=X_total, y=y_total)
print(f"{X_train.shape = }")
print(f"{y_train.shape = }")
print(f"{X_test.shape = }")
print(f"{y_test.shape = }")

X_train.shape = torch.Size([1626, 16])
y_train.shape = torch.Size([1626, 3])
X_test.shape = torch.Size([698, 16])
y_test.shape = torch.Size([698, 3])


### Global Model Training Parameters

In [19]:
LOSS_FUNCTION = loss.my_cagr_loss

MAXIMIZE_LOSS = True

BATCH_SIZE = 64
EPOCHS = 2_000
TEST_BATCH_SIZE = 128
LEARNING_RATE = 0.001

STORE = True

assert BATCH_SIZE <= X_train.shape[0]
assert TEST_BATCH_SIZE <= X_test.shape[0]

### Model Training Functions

## Model Training

In [20]:
INPUT_SHAPE = X_train.shape[-1]
OUTPUT_SHAPE = y_train.shape[-1]

print(f"Input Shape: {INPUT_SHAPE}, Output Shape: {OUTPUT_SHAPE}")

Input Shape: 16, Output Shape: 3


In [21]:
ALLOW_NEGATIVE_WEIGHTS = False

MAX_LEVERAGE_WEIGHT = 1.5
MIN_LEVERAGE_WEIGHT = (MAX_LEVERAGE_WEIGHT - 1) / (OUTPUT_SHAPE - 1)

MAX_WEIGHT = MAX_LEVERAGE_WEIGHT if ALLOW_NEGATIVE_WEIGHTS else 1.0
MIN_WEIGHT = -MIN_LEVERAGE_WEIGHT if ALLOW_NEGATIVE_WEIGHTS else 0.0

# MAX_WEIGHT = 1.5
# MIN_WEIGHT = None

print(f"Max. Weight: {MAX_WEIGHT}, Min. Weight: {MIN_WEIGHT}")

Max. Weight: 1.0, Min. Weight: 0.0


In [22]:
my_net = Net(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, layer_sizes=[8, 16, 8], 
             allow_negative_weights=ALLOW_NEGATIVE_WEIGHTS, max_weight=MAX_WEIGHT, min_weight=MIN_WEIGHT).to(device=DEVICE)
my_net

Net(
  (fc0): Linear(in_features=16, out_features=8, bias=True)
  (fc1): Linear(in_features=8, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc_output): Linear(in_features=8, out_features=3, bias=True)
)

In [23]:
print(f"Max. Weight: {my_net.max_weight}, Min. Weight: {my_net.min_weight}")

Max. Weight: 1.0, Min. Weight: 0.0


In [ ]:
ml_utils.train(net=my_net, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, loss_fn=LOSS_FUNCTION, 
               lr=LEARNING_RATE, batch_size=BATCH_SIZE, epochs=EPOCHS, maximize_loss=MAXIMIZE_LOSS, store=STORE,
               borrowing_costs=BORROWING_COSTS,
              )

2024-02-26 21:29:15.198 | INFO     | quantified_strategies.ml_utils:train:142 - Training: '1708982955'
100%|█████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 199.12it/s]
2024-02-26 21:29:15.411 | INFO     | quantified_strategies.ml_utils:train:162 - Epoch: 0 / 2000, Loss: 3.2115, Val Loss: 12.3727,Hit Rate: 54.43%, Val Hit Rate: 57.74%
100%|█████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 151.95it/s]
2024-02-26 21:29:15.652 | INFO     | quantified_strategies.ml_utils:train:162 - Epoch: 1 / 2000, Loss: 3.3891, Val Loss: 12.5420,Hit Rate: 54.61%, Val Hit Rate: 57.59%
100%|█████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 172.69it/s]
2024-02-26 21:29:15.877 | INFO     | quantified_strategies.ml_utils:train:162 - Epoch: 2 / 2000, Loss: 3.6275, Val Loss: 12.7518,Hit Rate: 54.61%, Val Hit Rate: 57.74%
100%|████████████████████████████

## Visualise Neural Network

In [ ]:
n_sample = 1
training_data = False

viz_nn.visualize_layer_activations(
    fig=plt.figure(figsize=(12, 12)), 
    network=my_net,
    X=(X_train[n_sample] if training_data else X_test[n_sample]).reshape(-1, orig_X_total.shape[1]),
    y=y_train[n_sample] if training_data else y_test[n_sample],
    X_labels=orig_X_total.columns.tolist(),
    y_labels=ASSETS,
)

## Plot Loss

In [ ]:
MODEL_NAME = f"{Net.MODEL_TYPE}-model-latest"
plot_utils.create_loss_graph(model_name=MODEL_NAME)

In [ ]:
# raise ValueError

## Apply Asset Allocator

In [ ]:
LOAD_MODEL = False

In [ ]:
X_total_df, y_total_df = get_data(assets=ASSETS, is_classification=False)
X_train_df, X_test_df, y_train_df, y_test_df = ml_utils.split_data(X=X_total_df, y=y_total_df)

In [ ]:
if LOAD_MODEL:
    model = Net.load(input_shape=X_total_df.shape[1], output_shape=y_total_df.shape[1], name="latest")
else:
    model = my_net
model

In [ ]:
X_train_tensor, _ = ml_utils.convert_data_to_tensors(X=X_train_df, y=y_train_df)
X_test_tensor, _ = ml_utils.convert_data_to_tensors(X=X_test_df, y=y_test_df)

In [ ]:
def run(X_df: pd.DataFrame, y_df: pd.DataFrame, is_train: bool = True):
    # Convert data to tensor
    X_tensor, _ = ml_utils.convert_data_to_tensors(X=X_df, y=y_df)

    # Get allocation
    allocation = pd.DataFrame(model(X_tensor).detach().numpy(), index=y_df.index, columns=y_df.columns)

    # Get Strategy returns
    hodl_ret = y_df.loc[:, y_df.columns != CASH].mean(axis=1)
    strat_ret = (y_df * allocation).sum(axis=1)
    strat_pos = allocation.loc[:, allocation.columns != CASH].sum(axis=1).round(3)

    cum_hodl_ret = strategy_utils.get_cumulative_return(returns=hodl_ret, total=False)
    cum_strat_ret = strategy_utils.get_cumulative_return(returns=strat_ret, total=False)

    hodl_dd = strategy_utils.get_drawdown_statistics(returns=hodl_ret)["drawdown"]
    strat_dd = strategy_utils.get_drawdown_statistics(returns=strat_ret)["drawdown"]

    prop_cycle = plt.rcParams["axes.prop_cycle"]
    colors = prop_cycle.by_key()["color"]
    color_map = {"strategy": colors[0], "hodl": colors[1]}

    fig, ax = plt.subplots(figsize=(15, 7))

    ax.plot(cum_hodl_ret, label="HODL", color=color_map["hodl"])
    ax.plot(cum_strat_ret, label="Strategy", color=color_map["strategy"])
    ax.fill_between(
        cum_hodl_ret.index,
        0, cum_strat_ret - cum_hodl_ret,
        alpha=0.2, color="blue",
        label="Strategy - HODL",
    )
    ax.plot(hodl_dd, alpha=0.2, label="HODL: DD", color=color_map["hodl"])
    ax.plot(strat_dd, alpha=0.2, label="Strategy: DD", color=color_map["strategy"])

    if is_train:
        plt.title("Strategy: Training")
    else:
        plt.title("Strategy: Test")
    plt.legend(loc="upper left")
    plt.show()

    description = pd.concat([
        strategy_utils.describe(returns=strat_ret, pos=strat_pos, daily=True).to_frame(name="strategy"),
        strategy_utils.describe(returns=hodl_ret, daily=True).to_frame(name="hodl")
    ], axis=1)
    
    return description


In [ ]:
run(X_df=X_train_df, y_df=y_train_df, is_train=True)

In [ ]:
run(X_df=X_test_df, y_df=y_test_df, is_train=False)